In [1]:
import pandas as pd

In [2]:
c18=pd.read_excel('datasets/C-18.xlsx',skiprows=6,header=None,engine='openpyxl')

In [3]:
c13=pd.read_excel('datasets/C-13.xls',skiprows=7,header=None)

## obtain useful data from C-13 and C-18 for age-groups

- formulas used:

In [4]:
# STATE_NAMES=[list(np.unique(c18.iloc[:,2].values))]
STATE_NAMES=[]
for state in c18.iloc[:,2].values:
    if not (state in STATE_NAMES):
        STATE_NAMES.append(state)
AGE_GROUPS=list(c18.iloc[1:10,4].values)
# although it is a bit of manual work but it is worth the efforts
AGE_GROUP_RANGES=[list(range(5,10)),list(range(10,15)),list(range(15,20)),list(range(20,25)),list(range(25,30)),list(range(30,50)),list(range(50,70)),list(range(70,100))+['100+'],['Age not stated']]

In [5]:
useful_data=[]
for i,state in enumerate(STATE_NAMES):
    for j,age_grp in enumerate(AGE_GROUPS):
        # this list is to get only the years in the particular age-group

        true_false_list=[]
        for single_year_age in c13.iloc[:,4].values:
            if single_year_age in AGE_GROUP_RANGES[j]:
                true_false_list.append(True)
            else:
                true_false_list.append(False)

        # here i is the state code
        total_pop=c13[(c13.loc[:,1]==i) & (true_false_list)].iloc[:,5].values.sum()
        
        # tri
        tri_total=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]==age_grp) & (c18.iloc[:,3]=='Total')].iloc[0,8]

        item={
            'state-code':i,
            'state-name':state,
            'age-group':age_grp,
            'age-group-pop':total_pop,
            'tri-ratio':tri_total/total_pop,
        }

        useful_data.append(item)

In [6]:
df=pd.DataFrame(useful_data)

## age-analysis 

In [7]:
tri_list=[]
for i in range(36):
    values=df[df['state-code']==i].sort_values(by='tri-ratio',ascending=False).iloc[0,[2,4]].values
    tri_item={
        'state/ut':STATE_NAMES[i],
        'age-group':values[0],
        'percentage':round(values[1]*100,2)
    }

    tri_list.append(tri_item)

In [8]:
tri_df=pd.DataFrame(tri_list)

In [9]:
tri_df.to_csv('outputs/age-india.csv',index=False)

## observations

- clearly this result ismore diverse and interesting than previous version of the output, there we only had 30-39 age group as the dominating one in all states here we get real idea by percantages.
- now we are getting younger and older[w.r.t 30-49 of previous version] age groups as the dominating one
- in delhi we have 70+ intellectuals as the dominating and around 15% of them speak more than 3 langs

In [10]:
tri_df

,state/ut,age-group,percentage
0,INDIA,20-24,11.52
1,JAMMU & KASHMIR,20-24,29.58
2,HIMACHAL PRADESH,20-24,8.21
3,PUNJAB,15-19,45.76
4,CHANDIGARH,50-69,42.77
5,UTTARAKHAND,20-24,3.09
6,HARYANA,15-19,7.32
7,NCT OF DELHI,70+,15.00
8,RAJASTHAN,15-19,2.88
9,UTTAR PRADESH,20-24,2.44
